1 – The $a = const$ case: the mathematical problem

In [1]:
import sys 
import os   
sys.path.append(os.getcwd() + '/..')
import numpy as np
import math
import matplotlib.pyplot as plt 
import nm_lib as nm
from typing import Type
from matplotlib import animation
import matplotlib
matplotlib.use('TkAgg')

# --- Values reused in other tasks of ex_2 ---
x0 = -2.6 
xf = 2.6
pi = math.pi
# ---------------------------

def tfunc(xx):
    return math.cos(6.0 * pi * xx / 5.0) * math.cos(6.0 * pi * xx / 5.0) /  math.cosh(5.0 * xx * xx)

def discretisation(x0, xf, nump, dt_float: Type[float]):
    nint = nump - 1
    dh = dt_float((xf - x0) / nint)
    xx = dt_float(np.arange(nump) * dh + x0) 
    xx_half = dt_float(np.arange(nump) * dh + x0 + dh * 0.5)
    return nint, dh, xx, xx_half
    
nump = 65
nint, dh, xx, xx_half = discretisation(x0, xf, nump, dt_float=np.float64)

hh = np.zeros(nump)
for i in range(0, nump):
    hh[i] = tfunc(xx[i])

# Using pyplot for plotting
plt.plot(xx, hh, '-+', label='original function')
plt.xlabel("x")
plt.ylabel("u")
plt.legend(loc='upper right', shadow=False, fontsize='x-large')
plt.figtext(0.5, 0.01, "Original function with 65 grid points", ha='center')
plt.show()

2 – Spatial derivative.

_According to the results blow (using nm_lib.order_conv), it seems the accuracy of the backward difference is only 1st-order. However, it was shown previously the accuracy of the backward difference is 2nd-order._

_To explain this, first we need to note that the results from nm_lib.deriv_bck (or nm_lib.deriv_fwd) are defined on the half-points. While mesh refinement happens (double the grid points), the locations of half-points on different meshes are never the same (mislocated by $\frac{1}{2}\Delta x$). Therefore, the results of the order_conv function would be always 1st-order for variables defined on half-points._

_Nevertheless, this does not affect the "actual" or theoretical accuracy of equation (3), but the 2nd-order accurate derivatives would in fact represent the derivatives at half-points $x=x_{\frac{1}{2}}$, not $x=x_i$._

In [2]:
hp_numerical  = nm.deriv_bck(xx, hh, dtype=np.float64)

nump2 = 129
nint2, dh2, xx2, xx_half2 = discretisation(x0, xf, nump2, dt_float=np.float64)
hh2 = np.zeros(nump2)
for i in range(0, nump2):
    hh2[i] = tfunc(xx2[i])
hp_numerical2  = nm.deriv_bck(xx2, hh2, dtype=np.float64) 

nump4 = 257
nint4, dh4, xx4, xx_half4 = discretisation(x0, xf, nump4, dt_float=np.float64)
hh4 = np.zeros(nump4)
for i in range(0, nump4):
    hh4[i] = tfunc(xx4[i])
hp_numerical4  = nm.deriv_bck(xx4, hh4, dtype=np.float64)

plt.plot(xx_half4[1:nump4-1], hp_numerical4[1:nump4-1], 'x-', label='Numerical derivative')
plt.xlabel("x")
plt.ylabel("up")
plt.legend(loc='upper right', shadow=False, fontsize='x-large')
plt.figtext(0.5, -0.05, "Numerical derivatives with\n" \
            +str(nump4)+" grid points and "+str(np.float64), ha='center')
plt.show()

print("The order of accuracy of the backward difference method is", nm.order_conv(hp_numerical, hp_numerical2, hp_numerical4))

The order of accuracy of the backward difference method is 0.7507997583545996


3 – Time advance

In [3]:
CFL = 0.98
a = -1
dt, hh_step1 = nm.step_adv_burgers(xx, hh, a, CFL, nm.deriv_bck) 
hh_step1 = hh + hh_step1 * dt

plt.plot(xx[1:nump-1], hh[1:nump-1], 'x-', label='Initial solution')
plt.plot(xx[0:nump-1], hh_step1[0:nump-1], 'x-', label='One step temporal solution')
plt.xlabel("x")
plt.ylabel("u")
plt.legend(loc='upper right', shadow=False, fontsize='x-large')
plt.figtext(0.5, -0.05, "One step temporal solution of the linear advection equation\n" \
            "with "+str(nump)+" grid points", ha='center')
plt.show()

invalid command name "4856798080process_stream_events"
    while executing
"4856798080process_stream_events"
    ("after" script)
can't invoke "event" command: application has been destroyed
    while executing
"event generate $w <<ThemeChanged>>"
    (procedure "ttk::ThemeChanged" line 6)
    invoked from within
"ttk::ThemeChanged"


 4 – The boundaries

5 – Subsequent steps in time

6 – Comparison with the exact solution

_As mentioned previously, the derivatives at grid points are calculated using the forward difference method, and thus below we can see that the numerical solution is a bit shifted compared with the exact solution. More importantly, the amplitude of the numerical solution decreases over time, due to the numerical diffusion. Looking into the details, we can also find that the initial symmetric pulse becomes unsymmetric, which is caused by numerical dispersion._

In [4]:
def exactU(xx, hh, nt, a, CFL, bnd_limits, bnd_type: str = 'wrap'):
    if xx.size != hh.size:
        return None
    else:
        t = np.zeros(nt)
        unnt = np.zeros((xx.size, nt))   
        for i in range(0, nt - 1):
            dt = CFL * nm.cfl_adv_burger(a, xx)
            t[i+1] = t[i] + dt
            r = a * t[i+1]
            for j in range(0, xx.size):
                unnt[j, i] = tfunc((xx[j] - x0 - r) % (xf - x0) + x0) 
            tmp = unnt[bnd_limits[0]:hh.size-bnd_limits[1], i]
            unnt[:, i] = np.pad(tmp, bnd_limits, bnd_type) 
        return t, unnt

def compare(nump, nt, a, CFL, bnd_limits):
    nint, dh, xx, xx_half = discretisation(x0, xf, nump, dt_float=np.float64)

    hh = np.zeros(nump)
    for i in range(0, nump):
        hh[i] = tfunc(xx[i])

    t, unnt = nm.evolv_adv_burgers(xx, hh, nt, a, CFL, nm.deriv_fwd, 'wrap', bnd_limits)
    t_exact, unnt_exact = exactU(xx, hh, nt, a, CFL, bnd_limits, 'wrap')

    # Using pyplot for animation
    plt.figure()
    plt.xlim(xx[0], xx[nump-1])
    plt.ylim(unnt_exact.min() - 0.1 * (unnt_exact.max() - unnt_exact.min()), unnt_exact.max() + 0.1 * (unnt_exact.max() - unnt_exact.min()))
    plt.xlabel("x")
    plt.ylabel("u") 
    line1, = plt.plot([], [], label='Exact solution')
    line2, = plt.plot([], [], '--', label='Numerical solution')

    plt.legend(loc='upper right')
    time_text = plt.text(0.02, 0.95, '', transform=plt.gca().transAxes)
 
    def init():
        line1.set_data([], [])
        line2.set_data([], [])
        time_text.set_text('')
        return line1, line2
 
    def update(frame):
        line1.set_data(xx, unnt_exact[:,frame])  
        line2.set_data(xx, unnt[:,frame])   
        time_text.set_text("Time = " + str(t[frame]))
        return line1, line2
 
    anim = animation.FuncAnimation(plt.gcf(), update, init_func=init, frames=t.size-1, blit=True)
    plt.show(block=True)

bnd_limits = [0, 1]
compare(65, 100, -1, 0.98, bnd_limits)

invalid command name "5110113728process_stream_events"
    while executing
"5110113728process_stream_events"
    ("after" script)
can't invoke "event" command: application has been destroyed
    while executing
"event generate $w <<ThemeChanged>>"
    (procedure "ttk::ThemeChanged" line 6)
    invoked from within
"ttk::ThemeChanged"


7 – Resolution increase in space and time

_From two animations below, we can find that with higher grid resolutions, the numerical results are closer to the exact solution._

In [5]:
compare(129, 200, -1, 0.98, bnd_limits)
compare(257, 400, -1, 0.98, bnd_limits)

invalid command name "5133009280process_stream_events"
    while executing
"5133009280process_stream_events"
    ("after" script)
can't invoke "event" command: application has been destroyed
    while executing
"event generate $w <<ThemeChanged>>"
    (procedure "ttk::ThemeChanged" line 6)
    invoked from within
"ttk::ThemeChanged"
